# 자계추 hw1: Create dataset

- `compustat_permno`와 `CRSP_M` 사용
    - SAS 코드 따라가며 python으로 포팅. 
- 최종 결과인 `Assignment1_data` 를 만들기 
    - 최종 결과는 permno/date 순으로 정렬하여 first 25 obs 를 보일 것. 
    - month of December for year 1970, 1980, 1990, 2000, 2010에 대하여 아래를 report:
        - number of distinct permnos
        - mean/std/min/max of the monthly delisting-adjusted excess returns 


## Lecture Note에서 기억할 내용들

- Compustat vs CRSP
    - Compustat
        - id: `GVKEY`, `DATADATE`
        - owner: S&P Global 
    - CRSP
        - id: `PERMNO` (and `PERMCO`)
        - owner: University of Chicago Booth School of Business
    - `CCMXPF_LNKUSED` CCM 즉, merged table을 사용

## 가이드
- SAS log 확인하며 중간중간 단계에서 같은 결과가 나오는지 확인해라. 
    - shape check
- sample data는 정답지. 최종적으로 output이 일치하는지 확인. 
- SAS 를 파이썬으로 옮겨준 코드도 참고하기. 
    - summary statistics 등 뽑는거는 본인 코드 있으면 그거 쓰기. 

## 질문했던 것들

- long table vs wide table 
    - 왜 굳이 wide 안쓰고 long 써서 각종 문제가 생기게 하는지... permno를 1개만 만들어놓을 수 있다면 그냥 그걸 가지고 pivot table 하고나면 그 다음엔 ffill 등이 훨씬 용이해 짐. 
    - 이 wide를 하고 shift를 쓰는 것을 교수님도 말하심. missing date 찐빠가 날 일이 없음. 그냥 그 자리에 NaN이 차고 말지. 
    - 교수님이 말씀하시는 단점:
        - RDBMS 관점에서 비효율적임 
        - 테이블이 너무 많이 생김. 그 부분 비효율도 생각해라. 

## SAS --> Python 포팅

- SAS1: Connect WRDS from your PC
    - Get stock data (CRSP)
    - Get event data (CRSP)
    - Merge stock & event data
    - Remove duplicates (by permno, date)
    - House Cleaning
- SAS2: Define libs & macro variables
    - SAS 코드의 주석 참고. compustat 데이터에서 WHERE 로 조건 넣어 필터링함. 
        - 금융주 제외
        - standardized report만 쓰고 (?)
        - domestic report만 쓰고 
        - consolidated report (연결재무제표)만 쓴다. 

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

## Load datasets

In [37]:
CWD = Path('.').resolve()
DATA_DIR = CWD / 'data'

In [ ]:
CRSP_M_df = pd.read_csv(DATA_DIR / 'CRSP_M.csv')
compustat_df = pd.read_csv(DATA_DIR / 'compustat_permno.csv') 
sample_df = pd.read_csv(DATA_DIR / 'assignment1_sample_data.csv')

### CRSP

In [39]:
CRSP_M_df.columns

Index(['DATE', 'DLSTCD', 'PERMNO', 'SHRCD', 'EXCHCD', 'SICCD', 'DLRET',
       'PERMCO', 'PRC', 'VOL', 'RET', 'SHROUT', 'ALTPRC', 'rf'],
      dtype='object')

In [40]:
CRSP_M_df.shape

(2921193, 14)

In [41]:
CRSP_M_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921193 entries, 0 to 2921192
Data columns (total 14 columns):
 #   Column  Dtype  
---  ------  -----  
 0   DATE    int64  
 1   DLSTCD  float64
 2   PERMNO  int64  
 3   SHRCD   int64  
 4   EXCHCD  int64  
 5   SICCD   float64
 6   DLRET   float64
 7   PERMCO  int64  
 8   PRC     float64
 9   VOL     float64
 10  RET     float64
 11  SHROUT  float64
 12  ALTPRC  float64
 13  rf      float64
dtypes: float64(9), int64(5)
memory usage: 312.0 MB


In [43]:
CRSP_M_df['PERMNO'].nunique()

23088

In [132]:
CRSP_M_df['PERMCO'].nunique()

22567

In [133]:
# Is date-permno unique?
CRSP_M_df[['DATE', 'PERMNO']].duplicated().sum() # Yes

0

In [134]:
CRSP_M_df['EXCHCD'].unique() # 이미 필터는 처리 되어있다. 

array([ 1,  2,  3, 33, 32, 31], dtype=int64)

그래도 아래 따로 filter 구현. 

In [135]:
# filters

filter_common_stocks = [10, 11] # SHRCD
filter_exchange = [ # EXCHCD
    1, 31, # NYSE
    2, 32, # AMEX
    3, 33, # NASDAQ
]

plots

In [136]:
# TODO: Stock Exchange Composition을 groupby 사용하여 만들기. 별도 column에 NYSE, AMEX, NASDAQ, Other 표시
# TODO: Number of stocks 로 한 번, Market Cap으로 한 번 plot

In [137]:
# apply filters

CRSP_M_df = CRSP_M_df[ CRSP_M_df['SHRCD'].isin(filter_common_stocks) ]
CRSP_M_df = CRSP_M_df[ CRSP_M_df['EXCHCD'].isin(filter_exchange) ]

In [138]:
CRSP_M_df.shape

(2921193, 14)

In [139]:
CRSP_M_df.head()

,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf
0,19610131,NaN,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.25,0.0019
1,19610131,NaN,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.00,0.0019
2,19610131,NaN,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.75,0.0019
3,19610131,NaN,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.00,0.0019
4,19610131,NaN,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.50,0.0019


### compustat

In [140]:
compustat_df.columns

Index(['gvkey', 'datadate', 'itcb', 'pstk', 'pstkl', 'pstkrv', 'seq', 'txdb',
       'permno', 'permco'],
      dtype='object')

In [141]:
compustat_df.shape

(434269, 10)

In [142]:
compustat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434269 entries, 0 to 434268
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   gvkey     434269 non-null  int64  
 1   datadate  434269 non-null  int64  
 2   itcb      353075 non-null  float64
 3   pstk      382566 non-null  float64
 4   pstkl     383753 non-null  float64
 5   pstkrv    381640 non-null  float64
 6   seq       375039 non-null  float64
 7   txdb      358326 non-null  float64
 8   permno    264450 non-null  float64
 9   permco    264450 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 33.1 MB


In [143]:
compustat_df['gvkey'].nunique()

35553

In [144]:
compustat_df['permno'].nunique()

23853

In [145]:
compustat_df['permco'].nunique()

23349

datadate는 fiscal year end date이다. 

In [146]:
# Is date-permno unique?
compustat_df[['datadate', 'permno']].duplicated().sum() # No

169195

In [147]:
# Is date-gvkey unique?
compustat_df[['datadate', 'gvkey']].duplicated().sum() # No

## 수업시간에 다룬 내용. non-unique한 이유는 기업이 fiscal year을 바꾸거나 할 경우 두 데이터가 동시에 존재할 수 있기 때문이다. 


2089

In [148]:
# compustat_df.dropna(subset=['permno'], inplace=True) 
# permno 없는 row 여기서 삭제하는게 맞으나, 이걸 해주면 아래에서 row 수가 달라져서 검증 불가하기에 일단 놔둠. 

In [149]:
compustat_df.head()

,gvkey,datadate,itcb,pstk,pstkl,pstkrv,seq,txdb,permno,permco
0,1000,19611231,0.0,NaN,0.0,NaN,NaN,0.000,NaN,NaN
1,1000,19621231,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,1000,19631231,0.0,0.0,0.0,0.0,0.553,0.008,NaN,NaN
3,1000,19641231,0.0,0.0,0.0,0.0,0.607,0.020,NaN,NaN
4,1000,19651231,0.0,0.0,0.0,0.0,0.491,0.000,NaN,NaN


Null인 데이터가 꽤 보인다. 

CRSP, Compustat date 를 살펴보자

In [150]:
CRSP_M_df['DATE'].sample(10)

1475444    19910531
1268333    19880630
2902577    20120731
2292368    20010731
1309082    19890131
2754409    20090630
464425     19751031
2750292    20090529
483752     19760227
582008     19771031
Name: DATE, dtype: int64

In [151]:
compustat_df['datadate'].sample(10) # fiscal end date

89193     19811231
205311    19841231
86059     19781231
108055    19781231
83682     19771231
43984     19811130
46767     19781231
19692     19660831
407352    20041231
190221    19721231
Name: datadate, dtype: int64

## SAS 3

Construct BE data

### Merge CRSP-Compustat using CCM

- pk
    - crsp: [DATE, PERMNO]
    - compustat: [datadate, gvkey]
        - compustat 테이블에 ccm을 통해 생성한 permno 있음. 이를 기준으로 join

** 질문: 왜 (inner join 안쓰고) LEFT JOIN 쓰는지?  right table인 CRSP에 데이터 없으면 분석 불가한거 아닌가? **

```
* Add permno and permco to BE data using the link-used table;
* The nobs might increase because a firm can be matched to multiple permno's; 
proc sql; 
 create table compustat_permno  
 as select distinct a.*, b.upermno as permno, b.upermco as permco  
 from compustat as a 
 left join my_lib.ccmxpf_lnkused  
  ( keep = ugvkey upermno upermco ulinkdt ulinkenddt usedflag ulinktype  
  where = (usedflag = 1 and ulinktype in ("LU","LC")) ) as b 
 on a.gvkey = b.ugvkey 
 and (b.ulinkdt <= a.datadate or missing(b.ulinkdt) = 1) 
 and (a.datadate <= b.ulinkenddt or missing(b.ulinkenddt) = 1) 
 order by a.datadate, a.gvkey; 
quit;
proc sort data = compustat_permno; by gvkey datadate; run;
```

위 merge는 지금 주어진 CRSP, Compustat 테이블로 한게 아님. 

In [152]:
df = pd.merge(
    left=compustat_df, 
    right=CRSP_M_df, 
    left_on=['datadate', 'permno'], 
    right_on=['DATE', 'PERMNO'],
    how='left',
    )

In [153]:
df.sort_values(by=['gvkey', 'datadate'], inplace=True)

In [154]:
df[ ['permno', 'datadate'] ].duplicated().sum() # compustat쪽 

169195

In [155]:
df[ ['DATE', 'PERMNO'] ].duplicated().sum() # crsp쪽. merge 전엔 중복이 없었는데, merge 후 중복이 생겼다.

277527

In [156]:
(df['DATE'] == df['datadate']).sum()

156741

In [157]:
df['DATE'].isnull().sum()

277528

In [158]:
df['datadate'].isnull().sum()

0

In [159]:
df.shape # NOTE: Table WORK.COMPUSTAT_PERMNO created, with 434269 rows and 10 columns.

(434269, 24)

```SAS
* Calculate BE; 
data BE; 
set compustat_permno (where = (missing(permno) = 0)); 
year = year(datadate); 
```

In [160]:
len( df.dropna(subset=['permno'], inplace=False) ) # left에 대해서만 수행해주면 됨. right는 애초에 없으면 붙지 않았음. 

    # NOTE: There were 264450 observations read from the data set
    #   WORK.COMPUSTAT_PERMNO.
    #   WHERE MISSING(permno)=0;

264450

In [161]:
# 위를 보면 그냥 permno만 dropna 해주는게 숫자가 맞는 것 같은데... 
# datadate, DATE, PERMNO가 null인 경우도 빼줘야 하는거 아닌가? 

df.dropna(subset=['permno'], inplace=True)

In [162]:
# 이 부분, join 문제라 생각해 내가 임의로 넣은 부분임. 이게 문제인가? 

# df.dropna(subet=['datadate', 'DATE', 'permno', 'PERMNO'], how='any', inplace=True) # key가 없는 row들 삭제

In [163]:
len(df)

264450

In [164]:
# 날짜 만들기 
df['YEAR'] = df['DATE'] // 10000 # int로 된 연도
df['pd_DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d') # 원래 SAS코드에는 없는, pd용 datetime


In [165]:
df.columns

Index(['gvkey', 'datadate', 'itcb', 'pstk', 'pstkl', 'pstkrv', 'seq', 'txdb',
       'permno', 'permco', 'DATE', 'DLSTCD', 'PERMNO', 'SHRCD', 'EXCHCD',
       'SICCD', 'DLRET', 'PERMCO', 'PRC', 'VOL', 'RET', 'SHROUT', 'ALTPRC',
       'rf', 'YEAR', 'pd_DATE'],
      dtype='object')

```
if missing(ITCB) then ITCB = 0; * investment tax credit; 
```

In [166]:
# ITCB(Investment Tax Credit Balance): 없는 경우 0으로
# 이건 없는 경우가 많다고 함. 없는 회사를 다 뺄 수는 없으니 0으로. 

df['itcb'] = df['itcb'].fillna(0)

```
BVPS = PSTKRV; * preferred stock - redemption value; 
if missing(BVPS) then BVPS = PSTKL; * preferred stock - liquidating value; 
if missing(BVPS) then BVPS = PSTK; * preferred stock- par value; 
if missing(BVPS) then BVPS = 0; 
```

BE = SEQ + TXDB + ITCB - BVPS 를 위해 BVPS를 구하는데, 

여기서 BVPS에 많은 처리가 들어간다. 우선주의 가치를 어떻게 산정해야 하지? 

1. PSTKRV, preferred stock의 redemption value가 있다면 이걸로. 
    - redemption value: 회사가 자진상장폐지 등의 이유로 주식을 재매입할 때의 금액
2. 그게 없으면 PSTKL, liquidating value로 
3. 또 없으면 PSTK, par value로 
4. 다 없으면 0으로 우선주 가치를 판정

이런 식의 operation이 앞으로도 계속 나옴. 

뭐가 available하면 뭐를 쓰고... 그게 안되면 이러저러한 조건일 때 저걸 쓰고 등등.. 

이걸 매번 일일이 만들면 너무 힘드므로 처리 가능한 함수를 만들겠음. 

하지만 조건이 까다로움

- 우선순위를 정해 list로 넣을 수 있어야. 
- 가장 간단하게, x가 없으면 y를 쓴다 는 같은 row 내에서 가능 (추후 row apply하면 됨)
- 조건이 달릴 경우. 같은 row 내에서 x가 없으면 A일 때 y를 쓴다 는 식의 로직 처리 가능해야
- ts 방향으로도 fill이 가능해야 함. ffill 처럼. 이 경우 wide 형식의 panel data인 경우 편하게 할 수 있지만 long data의 경우일 때 처리 가능해야 함. 
    - groupby ffill하면 가능함. 
    - groupby 전 permno-date로 sort되어있어야 함. 

구체적으로 
- output
    - 원래의 df 형태를 유지한 채, 빈 곳의 값들이 채워져 나와야 한다. 
- input
    - 원래의 df
    - 그 df에서 채울 대상
    - row 로직으로 채울껀지 
    - ts 로직으로 채울껀지
- row-wise logic
    - If row['target'] is empty, 
    - Additional condition
    - Fill something
- ts-wise logic
    - ts series만들어놓고 
    - if row['target'] is empty, 
    - Additional condition
    - Fill pre-made ts series

In [167]:
from abc import ABC, abstractmethod

class FillLogic(ABC):
    def __init__(self, target_col):
        self.target_col = target_col
    
    def check_empty(self, row):
        # return row[self.target_col] is np.nan
        return pd.isna(row[self.target_col])
    
    def run(self, row):
        if self.check_empty(row):
            return self.fill(row)
        else:
            return row[self.target_col]
    
    @abstractmethod
    def fill(self, row):
        raise NotImplementedError


In [168]:
class FillZero(FillLogic):
    def __init__(self, target_col):
        super().__init__(target_col)

    def fill(self, row):
        return 0

class FillReplace(FillLogic):
    def __init__(self, target_col, replace_col):
        super().__init__(target_col)
        self.replace_col = replace_col 

    def fill(self, row):
        return row[self.replace_col]

In [169]:
df.columns

Index(['gvkey', 'datadate', 'itcb', 'pstk', 'pstkl', 'pstkrv', 'seq', 'txdb',
       'permno', 'permco', 'DATE', 'DLSTCD', 'PERMNO', 'SHRCD', 'EXCHCD',
       'SICCD', 'DLRET', 'PERMCO', 'PRC', 'VOL', 'RET', 'SHROUT', 'ALTPRC',
       'rf', 'YEAR', 'pd_DATE'],
      dtype='object')

In [170]:
df['bvps'] = df['pstkrv']

돌리기 전

** 질문: 애초에 pstkrv에 -가 있는데, 빼주고 시작해야하지 않나?  **

마지막에 - BVPS 해주니까, 이 경우 - 값들이 다 +로 바뀌면서 더해지는 경우가 생길텐데???

In [171]:
df['bvps'].describe()

count    247399.000000
mean         21.584027
std         388.500150
min         -67.000000
25%           0.000000
50%           0.000000
75%           0.000000
max       81248.000000
Name: bvps, dtype: float64

In [172]:
df['bvps'].isnull().sum()

17051

돌린 후

In [173]:
fill_pstkl = FillReplace('bvps', 'pstkl').run
fill_pstk = FillReplace('bvps', 'pstk').run
fill_zero = FillZero('bvps').run


In [174]:
df['bvps'] = df.apply(lambda row: fill_pstkl(row), axis=1)
df['bvps'] = df.apply(lambda row: fill_pstk(row), axis=1)
df['bvps'] = df.apply(lambda row: fill_zero(row), axis=1)

In [175]:
df['bvps'].isnull().sum()

0

In [176]:
df['bvps'].describe()

count    264450.000000
mean         20.304002
std         376.690016
min         -67.000000
25%           0.000000
50%           0.000000
75%           0.000000
max       81248.000000
Name: bvps, dtype: float64

```
BE = SEQ + TXDB + ITCB - BVPS; * If SEQ or TXDB is missing, BE, too, will be missing; 

if BE<=0 then BE = .; * If BE<0, the value of BE is taken to be missing;  

label datadate = "Fiscal Year End Date"; 
keep gvkey datadate year BE permno permco; 
run;
```

In [177]:
df['be'] = df['seq'] + df['txdb'] + df['itcb'] - df['bvps']

In [178]:
df['be'].isnull().sum()

42366

In [179]:
df.loc[ df['be'] <= 0, 'be' ] = 0

fiscal year != calendar year이기 때문에, 

기업이 fiscal year을 바꿀 경우 한 calendar year에 두 결과값이 나오는 경우들이 있다. 

이 경우 given calendar year에서 가장 뒤에 있는 데이터를 사용

** 질문: gvkey랑 permco랑 안맞는 상황. 그래도 sort by gvkey, permno로 해도 되는지... ** 

현재 한 date에 company가 유일하지 않음. 

In [180]:
gvkey_permco = df.groupby(['datadate', 'gvkey'])['permco'].nunique()

In [181]:
gvkey_permco[ gvkey_permco > 1 ] 
# 위에서 싹다 dropna 처리해주면 없어지는데...안하면 이렇게 남아있음. 

datadate  gvkey
19851231  7276     2
19861231  7276     2
19991231  28883    2
          30331    2
20001231  28883    2
20011231  28883    2
20021231  28883    2
20030630  13312    2
20031231  28883    2
20040630  13312    2
20041231  28883    2
20050630  13312    2
20051231  28883    2
20060630  13312    2
20061231  28883    2
20070630  13312    2
20071231  28883    2
20080630  13312    2
20090630  13312    2
20100630  13312    2
20110630  13312    2
20120630  13312    2
Name: permco, dtype: int64

In [182]:
permco_gvkey = df.groupby(['datadate', 'permco'])['gvkey'].nunique()
permco_gvkey[ permco_gvkey > 1 ] # 이건 있다. 이건 말이 되나? 

datadate  permco 
19851231  20799.0    3
19861231  20799.0    3
19871231  20799.0    3
19881231  20799.0    3
19891231  20799.0    3
                    ..
20091231  41998.0    3
20101231  41998.0    3
20111231  53441.0    3
20121231  41998.0    2
          53441.0    3
Name: gvkey, Length: 103, dtype: int64

```
* In some cases, firms change the month in which their fiscal year ends,  
* resulting in two entries in the Compustat database for the same calendar year y.  
* In such cases, data from the latest in the given calendar year y are used.;  
proc sort data = BE; by gvkey permno year datadate; run; 
data BE; 
 set BE; 
 by gvkey permno year datadate; 
 if last.year; 
run; 
proc sort data = BE nodupkey; by gvkey permno year datadate; run;
```

In [183]:
df.sort_values(by=['gvkey', 'permno', 'YEAR', 'datadate',], inplace=True)

In [184]:
len(df)

264450

In [185]:
df = df.groupby(['gvkey', 'permno', 'YEAR', 'datadate']).last().reset_index()

In [186]:
len(df) # NOTE: The data set WORK.BE has 263854 observations and 6 variables.

# TODO: 잘못나온다. 너무 많이 짤렸다. 156741

156741

## SAS 5

Construct ME and return data (delisting adjusted)

### delisting returns

In [12]:
CRSP_M_df

,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf
0,19610131,NaN,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.2500,0.0019
1,19610131,NaN,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.0000,0.0019
2,19610131,NaN,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.7500,0.0019
3,19610131,NaN,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.0000,0.0019
4,19610131,NaN,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.5000,0.0019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921188,20121231,574.0,76999,11,3,7372.0,-0.765517,11056,NaN,123365.0,NaN,6855.0,0.3120,0.0001
2921189,20121231,580.0,93007,11,3,9999.0,-0.774834,53201,NaN,121619.0,NaN,57097.0,0.6307,0.0001
2921190,20121231,584.0,38790,11,2,1311.0,-0.762470,1933,NaN,21350.0,NaN,19048.0,0.3321,0.0001
2921191,20121231,584.0,89761,11,2,3714.0,2.520000,44123,NaN,39636.0,NaN,7107.0,0.3700,0.0001


In [ ]:
def process_delisting_returns(row):
    DLRET = row['DLRET']
    DLSTCD = row['DLSTCD']

    loss30_codes = [500, 520] + list(range(551, 574)) + [574, 580, 584] # -30%, other values는 -100%
    # TODO: 하다 말고 잔다. 이어서 하기. 

## SAS 6

Merge BE and ME with return data